# Linear Regression in Action!


Предскажем цену съёмной квартиры в Нью-Йорке. Датасет: https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data


## 0. Описание задания 

Начнём наш крестовый поход за дешёвой недвижимостью с предобработки данных. 

In [ ]:
PATH = "~/PycharmProjects/Datasets/"
hw = pd.read_csv(PATH + 'homework.txt', sep=".", encoding='CP1251') 
hw

In [ ]:
hw.iloc[9, 0]

## 1. Предобработка данных 

In [1]:
import pandas as pd               
import numpy as np                
import matplotlib.pyplot as plt   
import seaborn as sns             

plt.rcParams["figure.figsize"] = [14, 8]
%config Completer.use_jedi = False
%matplotlib inline

# import warnings
# warnings.filterwarnings("ignore")

print("Imports Done!")

Imports Done!


In [2]:
df = pd.read_csv('/home/aleksey/PycharmProjects/Datasets/airbnb_ny/AB_NYC_2019.csv') 
print('Размер выборки:', df.shape)                         
df.head() 

Размер выборки: (48895, 16)


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [6]:
df.describe()

,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
count,4.889500e+04,4.889500e+04,48895.000000,48895.000000,48895.000000,48895.000000,48895.000000,38843.000000,48895.000000,48895.000000
mean,1.901714e+07,6.762001e+07,40.728949,-73.952170,152.720687,7.029962,23.274466,1.373221,7.143982,112.781327
std,1.098311e+07,7.861097e+07,0.054530,0.046157,240.154170,20.510550,44.550582,1.680442,32.952519,131.622289
min,2.539000e+03,2.438000e+03,40.499790,-74.244420,0.000000,1.000000,0.000000,0.010000,1.000000,0.000000
25%,9.471945e+06,7.822033e+06,40.690100,-73.983070,69.000000,1.000000,1.000000,0.190000,1.000000,0.000000
50%,1.967728e+07,3.079382e+07,40.723070,-73.955680,106.000000,3.000000,5.000000,0.720000,1.000000,45.000000
75%,2.915218e+07,1.074344e+08,40.763115,-73.936275,175.000000,5.000000,24.000000,2.020000,2.000000,227.000000
max,3.648724e+07,2.743213e+08,40.913060,-73.712990,10000.000000,1250.000000,629.000000,58.500000,327.000000,365.000000


In [3]:
df.info() # Посмотрим на информацию по типам переменных 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              48895 non-null  int64  
 1   name                            48879 non-null  object 
 2   host_id                         48895 non-null  int64  
 3   host_name                       48874 non-null  object 
 4   neighbourhood_group             48895 non-null  object 
 5   neighbourhood                   48895 non-null  object 
 6   latitude                        48895 non-null  float64
 7   longitude                       48895 non-null  float64
 8   room_type                       48895 non-null  object 
 9   price                           48895 non-null  int64  
 10  minimum_nights                  48895 non-null  int64  
 11  number_of_reviews               48895 non-null  int64  
 12  last_review                     

In [4]:
df.isnull().sum()  # посмотрим на то есть ли в переменных пропуски

id                                    0
name                                 16
host_id                               0
host_name                            21
neighbourhood_group                   0
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
last_review                       10052
reviews_per_month                 10052
calculated_host_listings_count        0
availability_365                      0
dtype: int64

Пропусков нет. Это хорошая новость. Посмотрим как выглядит распределение цен. 

In [ ]:
# df.price.hist(bins=30);
sns.histplot(df.price, bins=50)

У распределения цен есть проблема - очень длинный хвост. Такой вид распределения называется логнормальным, поскольку если его прологарифмировато, то оно станет похоже на нормальное. 

Пути борьбы с выбросами: 

https://towardsdatascience.com/ways-to-detect-and-remove-the-outliers-404d16608dba

https://medium.com/swlh/identify-outliers-with-pandas-statsmodels-and-seaborn-2766103bf67c

https://stackoverflow.com/questions/23199796/detect-and-exclude-outliers-in-pandas-data-frame

In [ ]:
sns.boxplot(data=df, y=df['price'])

Исходя из графика видно - у нас очень много выбросов. 

В выборке встречаются квартиры с довольно большой стоимостью. Такие наблюдения называются выбросами. С ними нужно бороться, иначе наша модель подстроится под них, а это плохо. Проще говоря - модель переобучится. Сгладим распределение цен, прологарифмировав его. Так довольно часто поступают с целевой переменной. 

In [ ]:
df['price'] = np.log(df['price'])

In [ ]:
sns.histplot(df.price, bins=30)

In [ ]:
sns.boxplot(data=df, y=df['price'])

Посмотрим на рспределение всех остальных признаков.

In [ ]:
df.drop('price',axis=1).hist(figsize=(20, 12));

Выбросов в остальных данных не так уж и много. Значит можно использовать линейную регрессию. 

Кроме гистограммок имеет смысл взглянуть на матрицу корреляций. 

In [ ]:
plt.figure(figsize=(14, 10))
sns.heatmap(df.corr(), annot=True)

In [ ]:
plt.figure(figsize=(14, 10))
sns.heatmap(df.corr()[abs(df.corr()) > 0.5], annot=True)

In [ ]:
from scipy import stats

cols = ['totsp', 'livesp', 'kitsp', 'dist', 'metrdist', 'walk',
       'brick', 'floor', 'code']

for col in cols: 
    pearson_coef, p_value = stats.pearsonr(df[col], df['price'])
    print("PearsonR между {} и price {} с p-значением pvalue = {}".format(col, pearson_coef, p_value ))
    
    if p_value < 0.001:
        print('Корреляция между {} и price статистически значимая'.format(col))
    elif p_value < 0.05:
        print('Корреляция между {} и price средняя'.format(col))
    elif p_value < 0.1:
        print('Корреляция между {} и price слабая'.format(col))
    else:
        print('Корреляция между {} и price статистически незначимая'.format(col))
        
    if pearson_coef > 0 :
        if pearson_coef > 0.85:
            print('Коэффициент ~{} показывает положительную очень сильную связь\n'.format(pearson_coef))
        elif pearson_coef > 0.75 :
            print('Коэффициент ~{} показывает положительную достаточно сильную связь\n'.format(pearson_coef))
        elif pearson_coef > 0.60:
            print('Коэффициент ~{} показывает положительную относительно сильную связь\n'.format(pearson_coef))
        elif pearson_coef > 0.50 :
            print('Коэффициент ~{} показывает положительную среднюю связь\n'.format(pearson_coef))
        else:
            print('Коэффициент ~{} показывает положительную слабую связь\n'.format(pearson_coef))
    else:
        if abs(pearson_coef) > 0.85:
            print('Коэффициент ~{} показывает негативную очень сильную связь\n'.format(pearson_coef))
        elif abs(pearson_coef) > 0.75 :
            print('Коэффициент ~{} показывает негативную достаточно сильную связь\n'.format(pearson_coef))
        elif abs(pearson_coef) > 0.60:
            print('Коэффициент ~{} показывает негативную относительно сильную связь\n'.format(pearson_coef))
        elif abs(pearson_coef) > 0.50 :
            print('Коэффициент ~{} показывает негативную среднюю связь\n'.format(pearson_coef))
        else:
            print('Коэффициент ~{} показывает негативную слабую связь\n'.format(pearson_coef))

Чисто логически переменные metrdist и walk должны коррелировать, однако по расчётам этого не происходит. 

Для более адекватной оценки стоило бы группировать датасет по переменной walk, однако из-за того что ЛР является интерпертируемой моделью, то из результатов работы модели эти зависимости можно будет получить в явном виде. 

Дополнительно посмотрим на попарные диаграммы рассеивания.

In [ ]:
# sns.pairplot(df)

Теперь распределение выглядит более приятно. 

Займёмся предобработкой категориальных переменных при помощи одного горячего кодирования (One Hot Encoding) (кек). При таком преобразовании категориальной переменной мы создаем столько новых столбцов, сколько различных значений этой переменной у нас было. Обычно, при этом, первый столбец убирается, чтобы не создавать линейно-зависимых столбцов. Например, если у переменной `"погода"` есть три состояния `"хорошая"`, `"средняя"` и `"нормальная"`, то после `OneHotEncoding` мы получим три столбца, где значения будут либо нулями, либо единицами, в зависимости от того, какая погода была в этом наблюдении.

## Важно
`OneHotEncoding` в общем случае тоже может привнести информацию о тестовой выборке в тренировочную в том случае, если в категориальном столбце в отложенной выборке могут содержаться новые значения. Если такая вероятность есть, то как и с другими методами препроцессинга, `OneHotEncoding` нужно обучать на трейне и использовать на тесте. 

В остальных случаях, можно воспользоваться удобной оберткой в `pandas` - `pd.get_dummies()`

In [ ]:
# Чтобы не произошло утечки данных из трейна в тест, то стоит использовать OHE из sklearn'а.

from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()

# .fit на трейне, .transform на тесте!
# .fit(train), .transform(test)

In [ ]:
# Сделали OHE для категориальной переменной
df_categor = pd.get_dummies(df['code'], drop_first=True, prefix='code')
# drop_first=True используется для того, чтобы избежать идеальной мультеколлинеарности в данных
# Следовательно - использовать обязательно!

# Объединили назад наши таблички
df = pd.concat([df.drop('code',axis=1), df_categor], axis=1)
df.head()

Делим данные на тренировочные и тестовые! $30\%$ данных откладываем для тестирования качества модели. Остальные $70\%$ берём для обучения. 

In [ ]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size = 0.3)

Последний предобрабатывательский штрих это скалирование (стандартизация непрерывных переменных). Вспомним зачем его делают. Обычно, когда обучают модель, хотят минимизировать ошибку, которую она допускает. Чаще всего эту функцию минимизируют численно. Если переменные измерены в разных шкалах (что-то в тоннах, что-то в годах и тд), алгоритм может при обучении заблудиться. Хорошо бы направить его и подтолкнуть в нужном направлении. Таким толчком является стандартизация переменных. 

Из каждой переменной вычетают среднее и делят на стандартное отклонение. Это очищает переменные от своих уникальных шкал и упрощает путь алгоритма к оптимальной точке. Обычно стандартное отклонение и среднее для скалирования оценивают на обучающей выборке. К тестовой применяют уже оценённый результат. Это позволяет не подглядывать в тестовую часть и не улучшать за счёт этого подглядывания прогнозы. Подглядывать - нечестно! 

Стандартизация нужна и для корректного применения регуляризации в линейной регрессии, иначе будут большие штрафы за коэффициенты, а это ухудшает качество модели.

In [ ]:
from sklearn.preprocessing import StandardScaler

# объявили скалировщик!
scaler = StandardScaler()

# учим скалировщик скалировать все переменны на трэйне
scaler.fit(df_train.loc[:, ['totsp', 'livesp', 'kitsp', 'dist', 'metrdist']]) 

In [ ]:
# Применяем скалировщик к трэйну
df_train_scale = scaler.transform(df_train.loc[:, ['totsp', 'livesp', 'kitsp', 'dist', 'metrdist']])

# Применяем скалирвощик к тесту 
df_test_scale = scaler.transform(df_test.loc[:, ['totsp', 'livesp', 'kitsp', 'dist', 'metrdist']])

In [ ]:
# Заменяем значения на отскалированные
df_train.loc[:, ['totsp', 'livesp', 'kitsp', 'dist', 'metrdist']] = df_train_scale
df_test.loc[:, ['totsp', 'livesp', 'kitsp', 'dist', 'metrdist']] = df_test_scale

Все непрерывные переменные теперь выглядят проскалированными =) 

In [ ]:
df_train.head()

In [ ]:
print(df_train.shape) # Посмотрим на размеры трэйна и теста 
print(df_test.shape)  

In [ ]:
# Вытаскиваем цены и параметры квартир по разным переменным для удобства 

y_train = df_train.price 
y_test = df_test.price 

X_train = df_train.drop('price', axis=1).to_numpy()
X_test = df_test.drop('price', axis=1).to_numpy()

## 2. Константный прогноз 

Шаг первый. Построим константный прогноз. Будем говорить, что стоимость любой квартиры равна среднему значению. Это самый глупый прогноз, который мы можем сделать. Мы будем сравнивать с ним прогнозы более сложных моделей. Это бейзлайн-модель (baseline). Иногда, когда данные очень шумные, то она может быть лучше других моделей. Но это редкий вариант.

In [ ]:
y_mean = np.mean(y_train)                     # посчитали среднее 
y_pred_naive = np.ones(len(y_test)) * y_mean  # спрогнозировали им цену всех квартир в тестовой выборке
y_pred_naive[:5]

In [ ]:
np.exp(y_pred_naive[:5]) # стоимости квартир в тыс. $

## 3. Метрики качества для регрессии 

Мы сделали выше прогноз. Теперь мы хотим понять насколько он хороший. Для этого обычно используют метрики. Посмотрим на несколько таких метрик. 

In [ ]:
from sklearn import metrics  # подгружаем метрики

Первой метрикой, с которой мы познакомимся, будет MAE (mean absolute error), средняя абсолютная ошибка. Она вычисляется следующим образом: 

$$ MAE = \frac{1}{n}\sum_{i=1}^{n} |y_i - \hat{y}_i|. $$

Если мы спрогнозировали, что квартира стоит 20 рублей, а она стоила 10 рублей, мы ошиблись на |10 - 20| = 10 рублей. Средняя абсолютная ошибка - это средняя сумма рублей, на которую мы облажались. 

In [ ]:
metrics.mean_absolute_error(y_test, y_pred_naive)

Напомню, что мы прогнозируем логарифм цены, нам так удобнее. Ошибка выше считается в логарифмах. Если мы хотим посмотреть на ошибку в долларах, надо взять экспоненту от цен. Ниже мы можем увидеть, что в среднем ошибаемся на тридцать три с лишним тысячи долларов. (зависит от того как разобьётся датасет на тестовую и трейновую выборки)

In [ ]:
metrics.mean_absolute_error(np.exp(y_test), np.exp(y_pred_naive))

Второй метрикой является MSE (mean squared error), средняя квадратичная ошибка. Она вычисляется как 

$$ MSE = \frac{1}{n}\sum_{i=1}^{n} (y_i - \hat{y}_i)^2.$$

Смысл этой ошибки в том, чтобы штрафовать за большие ошибки сильнее, чем за маленькие. Если мы ошиблись на 5 долларов, то в ошибку войдёт 25. Если мы ошиблись на 10 долларов, то в ошибку войдёт 100. Чем выше ошибка, тем сильнее штраф. 

In [ ]:
metrics.mean_squared_error(y_test, y_pred_naive)

По аналогии перейдём к долларам. 

In [ ]:
metrics.mean_squared_error(np.exp(y_test), np.exp(y_pred_naive))

Получилось многовато! Всё дело в том, что это не просто доллары, это квадратные доллары. Мы же суммировали квадраты. Неплохо было бы вернутся к обычным долларам. Для этого надо взять из MSE квадратный корень. Тогда получится новая ошибка, RMSE - Root Mean Squared Error. 

In [ ]:
np.sqrt(metrics.mean_squared_error(np.exp(y_test), np.exp(y_pred_naive)))

Ошибка в среднем более чем на 57 тысяч долларов. Так как более большие ошибки входят с более большим весом, вполне логично, что RMSE получилось больше, чем MAE. 

Часто для нас принципиальным является не то, на сколько денег мы ошиблись, а то на сколько процентов мы ошиблись. Метрика, которая отлавливает процентную ошибку, называется MAPE (mean absolute percentage error), средняя абсолютная процентная ошибка. 

$$
MAPE = \frac{1}{n} \sum_{i=1}^n \frac{|y_i - \hat{y}_i|}{y_i}
$$

Она часто применяется в следующих задачах: например, вы прогнозируете спрос, и вам принципиально, на сколько процентов вы ошиблись, а не абсолютное значение. Если вы предсказали  один, а в реальности было  десять - это не то же самое, что вы предсказали  тысяча, а в реальности было  тысяча  девять. С точки зрения МАЕ или MSE, это две совершенно одинаковые ошибки. А если вас интересует, сколько в среднем на сколько процентов вы ошибаетесь, то это отражает МАРЕ.

Её нам придётся реальзовать самостоятельно. Благо, это не очень трудно. 

In [ ]:
def my_mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

my_mean_absolute_percentage_error(y_test, y_pred_naive)

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error

mean_absolute_percentage_error(y_test, y_pred_naive) * 100

Средняя ошибка примерно на $5\%$ от цены. 

Последняя метрика, с которой нам нужно познакомиться, это коэффициент детерминации, $R^2$. Он отражает то, какую долю дисперсии объясняемой переменной мы объяснили с помощью нашей модели:

$$ R^2 =1- \frac{ \frac{1}{n}\sum_{i=1}^{n} (y_i - \hat{y}_i)^2}{ \frac{1}{n}\sum_{i=1}^{n} (y_i - \bar{y}_i)^2} $$

Эту метрику очень сильно любят консалтеры и аудиторы, потому что только её они и знают. На самом деле в ней нет ничего хорошего. При добавлении в модель новых переменных она всегда растёт. У неё есть ещё несколько тонких математических недостатков, о которых вы можете узнать из книг. 

In [ ]:
metrics.r2_score(y_test, y_pred_naive)

Закинем все метрики в одну общую функцию, чтобы было удобно их печатать. 

In [ ]:
def print_metrics(y_test,y_pred):
    print('MAE:', metrics.mean_absolute_error(np.exp(y_test), np.exp(y_pred)))
    print('RMSE:', np.sqrt(metrics.mean_squared_error(np.exp(y_test), np.exp(y_pred))))
    print('R2:',  metrics.r2_score(y_test, y_pred))
    print('MAPE:', mean_absolute_percentage_error(y_test, y_pred) * 100)
    pass

In [ ]:
print_metrics(y_test, y_pred_naive)

## 3. Строим нашу первую регрессию!

Пришло время построить линейную регрессию! Эта модель говорит, что цена на квартиру формируется в результате суммирования тех характеристик, которыми она обладает с какими-то весами

$$\hat{y} = \beta_0 + \beta_1 x_1 + ... \beta_n x_n.$$

Например, если мы оценили модель и у нас получилось, что 

$$ price = 30 000 + 20 000 \cdot totsp,$$

то это означает, что средняя стоимость квартиры равна 30 тыс. долларам. При этом каждый дополнительный метр общей площади квартиры делает её дороже на 20 тысяч долларов. 

Для того, чтобы обучить регрессию минимизируют одну из метрик, перечисленных в прошлом разделе. В базовой комплектации регрессии это делают с MSE. Такая модель обладает огромным количеством няшных статистических свойств. 

In [ ]:
from sklearn.linear_model import LinearRegression

# Объявили модель
model_regression = LinearRegression()

# Обучили модель на тренировочной выборке 
model_regression.fit(X_train, y_train)

# Сделали прогнозы на тестовой выборке 
y_pred_regr = model_regression.predict(X_test)

Посмотрим на качество прогнозов. Мы стали ошибаться меньше, чем раньше. 

In [ ]:
print_metrics(y_test,y_pred_regr)

In [ ]:
model_regression.intercept_, model_regression.coef_

Посмотрим на то, какие признаки вносят в цену наибольший вклад. 

In [ ]:
feature_importance = pd.DataFrame({"feature": df.drop('price',axis=1).columns, 
                                  "importance": model_regression.coef_})

feature_importance.set_index('feature', inplace=True)
feature_importance.sort_values(["importance"], ascending=False, inplace=True)
plt.figure(figsize=(14, 10))
sns.barplot(data=feature_importance, y="importance", x=feature_importance.index)

## 4. Строим Lasso-регрессию 

Построим более сложную модель, LASSO-регрейссию. Фишка этой модели в том, что она зануляет лишние коэффиценты. 

In [ ]:
from sklearn.linear_model import Lasso

# Объявили модель
model_simplelasso = Lasso()

# Обучили модель на тренировочной выборке 
model_simplelasso.fit(X_train, y_train)

# Сделали прогнозы на тестовой выборке 
y_pred_lasso = model_simplelasso.predict(X_test)

Посмотрим на важность факторов для стоимости квартиры. 

In [ ]:
feature_importance = pd.DataFrame({"feature": df.drop('price',axis=1).columns, 
                                  "importance": model_simplelasso.coef_})

feature_importance.set_index('feature', inplace=True)
feature_importance.sort_values(["importance"], ascending=False, inplace=True)
plt.figure(figsize=(14, 10))
sns.barplot(data=feature_importance, y="importance", x=feature_importance.index)

In [ ]:
print_metrics(y_test,y_pred_lasso)

Важных факторов нет, всё занулилось. Метрики качества такие же, как при константном прогнозе. Почему такое произошло? Дело в том, что у модели есть гиперпараметр - сила зануления. И его нужно подбирать методом перебора. В нашей модели он стоял слишком большим. Давайте попробуем подобрать этот параметр. 

## 5. Подбор гиперпараметра для Lasso-регрессии

Будем делать перебор следующим способом: дробим тренировочную выборку на пять частей. На четырёх учим модель, на пятой прогнозируем. Смотрим на качество. И так по очереди выделяем для прогноза каждую из 5 частичек. Потом качество прогноза усредняем - кросс-валидация. Для какого параметра из решётки качетство получится наибольшим, тот мы и оставим. 

In [ ]:
from sklearn.model_selection import GridSearchCV

# Решётака для перебора параметра 
param_grid = {'alpha': [0.001, 0.01, 0.05, 0.1, 0.5, 0.8, 1]}

# Объявили модель 
model_lasso = Lasso() 

# Объявили перебор 
grid_cv_lasso = GridSearchCV(model_lasso, param_grid, cv = 5)
grid_cv_lasso.fit(X_train, y_train)
print('Лучшее значение параметра:', grid_cv_lasso.best_params_)

# Сделали прогнозы
y_pred_lasso = grid_cv_lasso.predict(X_test)

При параметре 0.001 качество у прогнозов получилось самым хорошим. Его и берём. Посмотрим на важность переменных

In [ ]:
grid_cv_lasso.best_estimator_.intercept_, grid_cv_lasso.best_estimator_.coef_

In [ ]:
feature_importance = pd.DataFrame({"feature": df.drop('price',axis=1).columns, 
                                  "importance": grid_cv_lasso.best_estimator_.coef_})

feature_importance.set_index('feature', inplace=True)
feature_importance.sort_values(["importance"], ascending=False, inplace=True)
plt.figure(figsize=(14, 10))
sns.barplot(data=feature_importance, y="importance", x=feature_importance.index)

И на качество модели. Оно оказывается близким к обычной регрессии. Судя по всему у нас в выборке нет лишних переменных и занулять нечего. 

In [ ]:
# Количество квартир в районах
df[[col for col in df.columns if "code" in col]].sum()

In [ ]:
print_metrics(y_test,y_pred_lasso)

In [ ]:
from sklearn.linear_model import LassoCV, RidgeCV, ElasticNetCV

lasso_cv = LassoCV(cv=5)
lasso_cv.fit(X_train, y_train)
print('Лучшее значение параметра:', lasso_cv.alpha_)

# Сделали прогнозы
y_pred_lasso_cv = lasso_cv.predict(X_test)

In [ ]:
feature_importance = pd.DataFrame({"feature": df.drop('price',axis=1).columns, 
                                  "importance": lasso_cv.coef_})

feature_importance.set_index('feature', inplace=True)
feature_importance.sort_values(["importance"], ascending=False, inplace=True)
plt.figure(figsize=(14, 10))
sns.barplot(data=feature_importance, y="importance", x=feature_importance.index)

In [ ]:
print_metrics(y_test, y_pred_lasso_cv)